In [8]:
# Import standard libraries
import time
# Import layer dependencies
from requests_html import AsyncHTMLSession
import numpy as np
import pandas as pd

In [9]:
# Config
# Create requests_html session
session = AsyncHTMLSession()

# Define the FantasySharks week Segment page numbers; this will need to be done once per season
season_start_week = 819
season_end_week = 836
weeks_in_season = 18

In [14]:
## Get Fantasy Sharks predictions
# Set up an empty dataframe to hold values
sharks = pd.DataFrame()

# List the position keys that Fantasy Sharks has assigned each player position
posList = ['1','2','4','5','7','15','6']
# Create a list of string values for each week in the season using the week key that Fantasy Sharks has assigned each week
weekList = [str(int(x)) for x in np.linspace(season_start_week, season_end_week, weeks_in_season)]

# Loop through positions and weeks
for posKey in posList:
    for weekKey in weekList:
        # Scrape each page
        # Define the url
        url = f'https://www.fantasysharks.com/apps/bert/forecasts/projections.php?League=&Position={posKey}&scoring=2&Segment={weekKey}&uid=4'
        # Scrape the page using requests_html to first render the JavaScript then read the html output and select the table needed
        r = await session.get(url)
        await r.html.arender()
        element = r.html.find('#toolData')[0].html

        # Transform the data
        # Convert the table to a dataframe
        df = pd.read_html(element)[0]
        # Filter for only those rows which contain data
        df = df.loc[df['#'].str.isnumeric()]
        # Create
        df['Week'] = int(weekKey) - season_start_week
        # Append the new data to the previous pages' data
        sharks = pd.concat([sharks, df], axis=0, ignore_index=True)
        # Wait 5 seconds to avoid timeout
        time.sleep(5)

/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_31778/1022996400.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(element)[0]
/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_31778/1022996400.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(element)[0]
/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_31778/1022996400.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(element)[0]
/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_31778/1022996400.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated 

In [16]:
sharks.head()

,#,Player,Tm,Opp,Att,Comp,Pass Yds,Pass TDs,0-9 Pass TDs,10-19 Pass TDs,...,50+ FGM,Miss,Punts,Punt Yds,Punts Inside 20,Yds Allowed,Pts Agn,Scks,DefTD,Safts
0,1,"Allen, Josh",BUF,ARI,33.4,21.3,246,1.6,0.6,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"Hurts, Jalen",PHI,GBP,29.1,19.0,213,1.4,0.4,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"Lawrence, Trevor",JAC,@MIA,40.8,26.8,286,1.5,0.4,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"Stroud, C.J.",HOU,@IND,34.6,21.9,266,1.5,0.7,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"Love, Jordan",GBP,@PHI,32.4,21.3,238,1.9,0.8,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Write to csv
sharks.to_csv('sharks.csv')
# This file can now be uploaded to s3